In [ ]:
import pandas as pd
import numpy as np

import json
import requests

from pathlib import Path
import os
import datetime

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [ ]:
# fetch FPL data online
fpl_orig_data = json.loads(requests.get('https://fantasy.premierleague.com/api/bootstrap-static/').text)
fpl_data = pd.DataFrame(fpl_orig_data['elements'])
fpl_data

In [ ]:
fpl_data[~fpl_data.chance_of_playing_next_round.apply(lambda x: np.isnan(x))]

In [ ]:
np.isnan(fpl_data.loc[0,'chance_of_playing_next_round'])

In [ ]:
# for some reason, fpl team numbers are otherwise alphabetic, except Leeds comes after Leicester...
teams = ['Arsenal', 'Aston Villa', 'Bournemouth', 'Brentford', 'Brighton',
        'Chelsea', 'Crystal Palace', 'Everton', 'Fulham', 'Leicester City', 'Leeds United',
         'Liverpool', 'Manchester City', 'Manchester Utd', 'Newcastle Utd',
       'Nottingham Forest', 'Southampton', 'Tottenham', 'West Ham','Wolves']

fpl_data['team'] = [teams[i-1] for i in fpl_data['team']]

In [ ]:
# change dtypes to numeric where appropriate
aux = []
for ix, col in fpl_data.items():
    aux.append(pd.to_numeric(col, errors='ignore'))
df = pd.DataFrame(aux).T

In [ ]:
df.groupby('team')['expected_goals_conceded'].sum() / 11

In [ ]:
df.groupby('team')['expected_goals'].sum()

In [ ]:
filepath = Path('../../../../season22_23/data/fpl_data/fpl_week_1.csv')
last_season_week1 = pd.read_csv(filepath)
last_season_week1.head()

In [ ]:
filepath = Path('../../../../season22_23/data/fpl_data/fpl_week_38.csv')
last_season_week38 = pd.read_csv(filepath, index_col=0)
last_season_week38.head()

In [ ]:
set(last_season_week38.columns).difference(set(last_season_week1.columns))

### 22-23 week 38 new data processing scheme

In [ ]:
# all data for season 22-23
data = []
for i in range(38):
    filepath = Path(f'../../../../season22_23/data/fpl_data/fpl_week_{i+1}.csv')
    gameweek_data = pd.read_csv(filepath, index_col=0)
    #modification_time = os.path.getmtime(filepath)
    #modification_datetime = datetime.datetime.fromtimestamp(modification_time)
    #gameweek_data['modification_datetime'] = modification_datetime
    data.append(gameweek_data)

df = pd.concat(data, ignore_index=True)
display(df.head())
display(df.tail())
display(df.shape)

In [ ]:
print('Percentage null:')
df.isnull().sum() / df.shape[0]

In [ ]:
df.name.nunique()

In [ ]:
df.id.nunique()

In [ ]:
df_grouped = df.groupby(['web_name', 'id']).count().reset_index()
df_grouped[df_grouped.id.duplicated()]

In [ ]:
df_grouped[df_grouped.id==159]

In [ ]:
df[df['id']==159]

In [ ]:
df.groupby(['web_name'])['points_per_game'].expanding(10).mean()

In [ ]:
df.groupby(['web_name'])['points_per_game'].rolling(10).mean()

In [ ]:
df['new_points'] = df.groupby(['web_name'])['total_points'].diff()
df[df.web_name=='Tierney'].head(20)[['web_name', 'total_points', 'new_points']]

# all data created last season